# I. GPU check

---



---



In [ ]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')

In [ ]:
if gpus :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus =  tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), 'Phyiscal GPUs,', len(logical_gpus), 'Logical GPUs')
    except RunTimeError as e :
        print(e)

# II. Imports & functions

---
---



In [ ]:
pip install plotly &> /dev/null

In [ ]:
pip install geoplot &> /dev/null

In [ ]:
pip install reverse_geocoder &> /dev/null

In [ ]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import time
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
import json
import ast
from geopy.distance import geodesic
import json
from scipy.stats import f_oneway

from urllib.request import urlopen
import reverse_geocoder as rg

import warnings
warnings.filterwarnings("ignore")

from google.colab import files
import plotly
# plotly.offline.init_notebook_mode(connected=True)

In [ ]:
# Function to give stats on missing values in a table or dataframe
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [ ]:
# Function to remove outliers from a dataframe's column
def remove_outliers(df, col1, col2, coef) :
  for value in df[col1].unique() :
    q1 = df[df[col1]==value][col2].quantile(0.25)
    q3 = df[df[col1]==value][col2].quantile(0.75)
    iqr = q3 - q1
    low = q1 - coef*iqr
    high = q3 + coef*iqr
    df[df[col1]==value] = df[df[col1]==value].loc[(df[df[col1]==value][col2] > low) & (df[df[col1]==value][col2] < high)]

In [ ]:
# Function to calculate distance from coordinates
def calculate_distance(row):
    origin_coords = (row['origin_lat'], row['origin_lng'])
    dest_coords = (row['destination_lat'], row['destination_lng'])
    return geodesic(origin_coords, dest_coords).meters

In [ ]:
# Function for string to tuple
def eval_value(string):
    return eval(string)

In [ ]:
# Function to get city and country from coordinates
# def getplace(lat, lon):
#     key = "key"
#     url = "https://maps.googleapis.com/maps/api/geocode/json?"
#     url += "latlng=%s,%s&sensor=false&key=%s" % (lat, lon, key)
#     v = urlopen(url).read()
#     j = json.loads(v)
#     components = j['results'][0]['address_components']
#     country = town = None
#     for c in components:
#         if "country" in c['types']:
#             country = c['long_name']
#         if "postal_town" in c['types']:
#             town = c['long_name']
#     return town, country

def getcountry(lat, lon):
  result = rg.search((lat, lon))
  return result[0]['cc']

def getreg(lat, lon):
  result = rg.search((lat, lon))
  return result[0]['admin1']

def getdep(lat, lon):
  result = rg.search((lat, lon))
  return result[0]['admin2']

def getcity(lat, lon):
  result = rg.search((lat, lon))
  return result[0]['name']

# III. Data loading & preprocessing

---



---





### 1. Data loading



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# routes_og = pd.read_csv("/content/drive/MyDrive/Work/Applications/Ynstant/routes.csv")
routes_og = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/routes.csv")
routes_og.columns

In [ ]:
routes_og.head()

### 2. Data pre-processing

In [ ]:
# Isolement des dimensions d'intérêt (on enlève polyline qui ne semble pas apporter des informations supplémentaires compréhensibles)

routes = routes_og[['_id', 'driver_uid', 'origin', 'destination', 'departure', 'arrival','length', 'reward', 'status', 'status_history']]

In [ ]:
# Transformation des colonnes de type object à des colonnes plus explicites

routes['origin'] = routes['origin'].apply(ast.literal_eval)
routes['destination'] = routes['destination'].apply(ast.literal_eval)
routes['reward'] = routes['reward'].apply(ast.literal_eval)

all_keys = set().union(*(d.keys() for d in routes['origin']))
for key in all_keys:
  routes[f'origin_{key}'] = routes['origin'].apply(lambda x: x.get(key))
routes.drop('origin', axis=1, inplace=True)

all_keys = set().union(*(d.keys() for d in routes['destination']))
for key in all_keys:
  routes[f'destination_{key}'] = routes['destination'].apply(lambda x: x.get(key))
routes.drop('destination', axis=1, inplace=True)

all_keys = set().union(*(d.keys() for d in routes['reward']))
for key in all_keys:
  routes[f'reward_{key}'] = routes['reward'].apply(lambda x: x.get(key))
routes.drop('reward', axis=1, inplace=True)

routes.head(2)

In [ ]:
# Transformation de reward_amount

all_keys = set().union(*(d.keys() for d in routes['reward_amount']))
for key in all_keys:
  routes[f'reward_{key}'] = routes['reward_amount'].apply(lambda x: x.get(key))
routes.drop('reward_amount', axis=1, inplace=True)

In [ ]:
# Copie en cas de besoin
routes_bis = routes.copy()

In [ ]:
# Transformation de status_history

routes['status_history'] = routes['status_history'].apply(eval_value)

created = []
in_progress = []
done = []
canceled = []

for i in range(len(routes['status_history'])):
  times = []
  statuses = []
  for d in routes['status_history'][i]:
    status = pd.Series(d['status'])
    time_update = pd.Timestamp(d['time_update'])
    times.append(time_update)
    statuses.append(status)
  if str(statuses[0][0]).lower() != 'created':
    print(i) #vérifier que le premier status est toujours = 'CREATED'
  if len(statuses) == 1:
    created.append(times[0])
    canceled.append(None)
    in_progress.append(None)
    done.append(None)
  if len(statuses) > 1 and str(statuses[-1][0]).lower() == 'canceled':
    created.append(times[0])
    canceled.append(times[-1])
    in_progress.append(None)
    done.append(None)
  if len(statuses) > 1 and str(statuses[-1][0]).lower() == 'done':
    created.append(times[0])
    canceled.append(None)
    in_progress.append(None)
    done.append(times[-1])
  if len(statuses) > 1 and str(statuses[-1][0]).lower() == 'in_progress':
    created.append(times[0])
    canceled.append(None)
    in_progress.append(times[-1])
    done.append(None)

# Création des timestamp de création ainsi que du dernier status

routes['created_time_update'] = pd.DataFrame(created)
routes['in_progress_time_update'] = pd.DataFrame(in_progress)
routes['done_time_update'] = pd.DataFrame(done)
routes['canceled_time_update'] = pd.DataFrame(canceled)

routes.drop('status_history', axis=1, inplace=True)
routes.head(2)

In [ ]:
routes.dtypes

In [ ]:
routes['length'] = routes['length'].astype(int)
routes['departure'] = pd.to_datetime(routes['departure'])
routes['arrival'] = pd.to_datetime(routes['arrival'])
routes['ride_rewarded'] = (routes['reward_value'].notnull()) & (routes['reward_value'] > 0)

In [ ]:
# Feature engineering

routes['trip_duration_min'] = (routes['arrival'] - routes['departure']).dt.total_seconds() / 60
routes['departure_day'] = routes['departure'].dt.date
routes['departure_hour'] = routes['departure'].dt.hour
routes['departure_dayofweek'] = routes['departure'].dt.dayofweek
routes['departure_month'] = routes['departure'].dt.month
routes['distance'] = routes.apply(calculate_distance, axis=1)
routes['trip_duration_bin'] = pd.qcut(routes['trip_duration_min'], 5, labels=['very_short', 'short', 'medium', 'long', 'very_long'])

In [ ]:
routes.head(2)

In [ ]:
# Copie en cas de besoin
routes_bis_bis = routes.copy()

In [ ]:
# Vérifications de données manquantes

missing_values_table(routes)

In [ ]:
# Description sommaire des données

routes.describe(include='all')

In [ ]:
# Omission des colonnes vides + destination_public_transport qui ne prend que la valeur []

routes = routes[['_id', 'driver_uid', 'departure', 'arrival', 'length', 'status',
       'origin_city', 'origin_lat', 'origin_address', 'origin_lng', 'origin_short_label', 'origin_public_transport',
       'destination_city', 'destination_lat', 'destination_address', 'destination_lng', 'destination_short_label',
       'trip_duration_min', 'trip_duration_bin', 'distance',
       'reward_status', 'reward_value', 'reward_currency', 'ride_rewarded',
       'departure_day', 'departure_hour', 'departure_dayofweek', 'departure_month',
       'created_time_update', 'in_progress_time_update', 'done_time_update', 'canceled_time_update']]

In [ ]:
routes.info()

In [ ]:
# Description sommaire des données

routes.describe(include='all')

> Nous constatons que les données ne contiennent pas de données manquantes, ni de doublons dans les ID des trajets.

> Nous remarquons aussi qu'il y a 57,447 conducteurs uniques, pour un total de 571,125 trajets, ce qui fait une moyenne de ~10 trajets par conducteurs.

> De la description, nous voyons déjà quelques statistiques intéressantes, comme par exemple le fait que le conducteur `mNSQAulMyrMkW8oCaC45xCDuPD92` fait partie des top utilisateurs, avec 1,066 trajets postés (nous creuserons les statistiques descriptives plus en détail dans le paragraphe suivant).

In [ ]:
# Vérification de la cohérences des données

print(routes[routes['departure'] > routes['arrival']])
print(routes[(routes['canceled_time_update'].notnull()) & (routes['status'].str.lower() != 'canceled')])
print(routes[(routes['in_progress_time_update'].notnull()) & (routes['status'].str.lower() != 'in_progress')])
print(routes[(routes['done_time_update'].notnull()) & (routes['status'].str.lower() != 'done')])
print("Correlation between Trip Length and Distance:", routes['length'].corr(routes['distance'])) # La distance et la durée sont corrélées, ce qui est très attendu

In [ ]:
fig = px.box(routes, x="status", y="trip_duration_min")
fig.show()

In [ ]:
routes[routes['trip_duration_min']> 7000][['origin_city', 'origin_lat', 'origin_address', 'origin_lng', 'origin_short_label', 'origin_public_transport','destination_city', 'destination_lat', 'destination_address', 'destination_lng', 'destination_short_label',]]

> Nous constatons qu'il y a des "outliers" dans les données. A titre d'exemple, il y a des trajets qui durent plus de 7000 minutes, i.e plus de 4 jours, dont un partant du Vietnam à Paris. Nous allons les garder car en théorie cela reste des trajets possibles (un trajet du Vietnam à Paris peut effectivement durer ~5 jours). Nous n'allons pas creuses ces cas plus en détail car ils ne sont pas nombreux comme on peut le voir sur les box plots de distribution.


> A part ceci, nous ne constatons pas d'anomalie dans les données jusque là (pas de données manquantes, ou incohérentes), nous allons donc procéder à l'analyse descriptive.

# IV. Descriptive analysis

> Avant de faire des analyses poussées, il est important de comprendre ce qui se passe. Ainsi, une analyse descriptive des faits et des patterns existants est nécessaire et constitue la partie la plus cruciale dans la construction des métriques impactantes d'un point de vue business.



### 1. Daily usage - Rides

In [ ]:
# Evolution de la moyenne des trajets quotidiens par mois

routes['month'] = pd.to_datetime(routes['departure_day']).dt.to_period('M')
rides_per_day_per_month = routes.groupby(['departure_day', 'month'])['_id'].nunique().reset_index()
rides_per_day_per_month.columns = ['departure day', 'month', 'number_of_rides']
average_daily_rides_per_month = round(rides_per_day_per_month.groupby('month')['number_of_rides'].mean().reset_index(name='average daily rides'),0)
average_daily_rides_per_month['month'] = average_daily_rides_per_month['month'].astype(str)

fig = px.line(average_daily_rides_per_month, x='month', y='average daily rides', title='Average Number of Daily Rides per Month')
fig.show()

In [ ]:
# Visualisation du nomber quotidien de trajets par status

rides_per_day_status = routes.groupby(['departure_day','status'])['_id'].nunique().reset_index()
rides_per_day_status.columns = ['departure day', 'ride status', 'number of rides']

fig = px.bar(rides_per_day_status, x='departure day', y='number of rides', color='ride status', title='Number of Rides by Day and Status', barmode='stack', category_orders={'ride status': ['CREATED','CANCELED','DONE','IN_PROGRESS']})
fig.show()

In [ ]:
# Visualisation du nomber quotidien de trajets par status (100% stacked bars)

pivot_rs = rides_per_day_status.pivot(index='departure day', columns='ride status', values='number of rides')
pivot_rs = round(pivot_rs.div(pivot_rs.sum(axis=1), axis=0) * 100,2)
rides_per_day_status_normalized = pivot_rs.reset_index().melt(id_vars='departure day', var_name='ride status', value_name='percentage')

fig = px.bar(rides_per_day_status_normalized, x='departure day', y='percentage', color='ride status', title='Number of Rides by Day and Status (Stacked to 100%)', barmode='stack', category_orders={'ride status': ['CREATED','CANCELED','DONE','IN_PROGRESS']})
fig.show()

> Entre 4.5k et 6k trajets sont effectués chaque jour, et ce depuis février 2024. Le nombre de trajets quotidiens est en augmentation constante: On est passé de 5,710 trajets en moyenne par jour en janvier 2024, à 6,161 trajets en moyenne par jour en février 2024 (+8%), versus 3,640 en décembre 2023 (+69%).

> Une grande partie des trajets quotidiens est finsalisée, notamment depuis Juillet 2023 (i.e le moment où le nombre de trajets a commencé à devenir significatif). Le nombre de trajets finalisés représente plus de 75% des trajets.

> Le nombre de trajets annulés a connu une légère augmentation depuis Novembre 2023, passant de ~20% à ~30% des trajets.

> Il y a des trajets qui ont le statut `CREATED`uniquement, un coup d'oeil sur la donnée montre que c'est des trajets qui ont été annulés mais dont l'ID a changé. L'ID des trajets n'est pas consistant, et un des points d'amélioration possibles d'un point de vue data serait de créer un ID plus consistant qui dépend de l'horaire du trajet, l'ID du client, et les coorodnnées de départ et de destination.



In [ ]:
# Visualisation du nombre quotidien de trajets par status de récompense

rides_per_day_reward = routes.groupby(['departure_day','ride_rewarded'])['_id'].nunique().reset_index()
rides_per_day_reward.columns = ['departure day', 'ride rewarded', 'number of rides']

fig = px.bar(rides_per_day_reward, x='departure day', y='number of rides', color='ride rewarded', title='Number of Rides by Day and Reward', barmode='stack')
fig.show()

In [ ]:
# Visualisation du nomber quotidien de trajets par status de récompense (100% stacked bars)

pivot_rr = rides_per_day_reward.pivot(index='departure day', columns='ride rewarded', values='number of rides')
pivot_rr = round(pivot_rr.div(pivot_rr.sum(axis=1), axis=0) * 100,2)
rides_per_day_reward_normalized = pivot_rr.reset_index().melt(id_vars='departure day', var_name='ride rewarded', value_name='percentage')

fig = px.bar(rides_per_day_reward_normalized, x='departure day', y='percentage', color='ride rewarded', title='Number of Rides by Day and Reward Status (Stacked to 100%)', barmode='stack')
fig.show()



> Entre 80% et 92% des trajets (jusqu'à 7.5k trajets par jour) sont sans passager.



In [ ]:
# Visualisation du nomber quotidien de trajets par longueur du trajet

bins = [0, 10, 30, 60, 120, float('inf')] # au vu de des quantiles de la durée des trajets (cf Description sommaire des données à la fin du paragraphe Data pre-processing), nous allons adapter la catégorisation pour qu'elle soit plus intuitive
routes['trip_duration_bin'] = pd.cut(routes['trip_duration_min'], bins=bins, labels=['very_short < 10min', 'short [10min ; 30min[', 'medium [30min ; 60min[', 'long [60min ; 120min[', 'very_long [120min ; 8,381min]'])

rides_per_day_length = routes.groupby(['departure_day','trip_duration_bin'])['_id'].nunique().reset_index()
rides_per_day_length.columns = ['departure day', 'trip duration category', 'number of rides']

fig = px.bar(rides_per_day_length, x='departure day', y='number of rides', color='trip duration category', title='Number of Rides by Day and Ride Length', barmode='stack')
fig.show()

In [ ]:
# Visualisation du nomber quotidien de trajets par longueur du trajet (100% stacked bars)

pivot_rl = rides_per_day_length.pivot(index='departure day', columns='trip duration category', values='number of rides')
pivot_rl = round(pivot_rl.div(pivot_rl.sum(axis=1), axis=0) * 100,2)
rides_per_day_length_normalized = pivot_rl.reset_index().melt(id_vars='departure day', var_name='trip duration category', value_name='percentage')

fig = px.bar(rides_per_day_length_normalized, x='departure day', y='percentage', color='trip duration category', title='Number of Rides by Day and Ride Length (Stacked to 100%)', barmode='stack')
fig.show()

> Quelques statistiques:

> *   40% à 50% des trajets (entre 3k et 3.7k trajets par jour) sont des trajets très courts (moins de 10 minutes)
> *   ~ 40% des trajets (~ 3k trajets par jours) sont des trajets courts (entre 10 et 30 minutes)
> *   10% à 15% des trajets quotidiens (~ 700-800 trajets par jour) sont moyens (entre 30 minutes et 1 heure)
> *  2% à 3% des trajets (~150 trajets par jour) sont longs (entre 1 et 2 heures)
> *  <= 3% des trajets (150 à 200 trajets par jour) sont très longs (plus de 2h, avec un maximum à 8,381 minutes = ~5 jours et 19 heures)

### 2. Daily usage - Users

In [ ]:
# Visualisation du nomber quotidien de conducteurs par status du conducteur

daily_activity = routes.groupby(['driver_uid', 'departure_day'])['ride_rewarded'].value_counts().unstack(fill_value=0).reset_index()
daily_activity.columns = ['driver_uid', 'departure_day', 'non_rewarded_rides_count', 'rewarded_rides_count']

daily_activity['driver_status'] = 'Rewarded'
daily_activity.loc[(daily_activity['rewarded_rides_count'] == 0) & (daily_activity['non_rewarded_rides_count'] > 0), 'driver_status'] = 'Non-Rewarded'
daily_activity.loc[(daily_activity['rewarded_rides_count'] > 0) & (daily_activity['non_rewarded_rides_count'] > 0), 'driver_status'] = 'Both'

users_per_status_day = daily_activity.groupby(['departure_day','driver_status'])['driver_uid'].nunique().reset_index(name='number of users')
users_per_status_day.columns = ['departure day', 'driver status', 'number of users']

fig = px.bar(users_per_status_day, x='departure day', y='number of users', color='driver status', title='Number of Drivers by Day and Status', barmode='stack', category_orders={'driver status': ['Both','Rewarded','Non-Rewarded']})
fig.show()

In [ ]:
# Visualisation du nomber quotidien de conducteurs par status du conducteur (100% stacked bars)

pivot_ds = users_per_status_day.pivot(index='departure day', columns='driver status', values='number of users')
pivot_ds = round(pivot_ds.div(pivot_ds.sum(axis=1), axis=0) * 100,2)
users_per_status_day_normalized = pivot_ds.reset_index().melt(id_vars='departure day', var_name='driver status', value_name='percentage')

fig = px.bar(users_per_status_day_normalized, x='departure day', y='percentage', color='driver status', title='Number of Drivers by Day and Status (Stacked to 100%)', barmode='stack', category_orders={'driver status': ['Both','Rewarded','Non-Rewarded']})
fig.show()

> En termes de conducteurs, on a atteint ~3k conducteur par jour en février 2024. En zoomant sur le type de trajet : **85% à 90% des conducteurs quotidiens (2.5k à 3k conducteur par jour) n'effectuent que des trajets récompensés, i.e sans passager**, 10% à 13% des conducteurs font les deux, et moins de 3% ne font que des trajets non récompensés.

In [ ]:
# Distribution du nombre de trajets par conducteur

rides_per_user = routes.groupby(['driver_uid'])['_id'].nunique().reset_index(name='number of rides')
months_per_user = routes.groupby(['driver_uid'])['month'].nunique().reset_index(name='number of months')

fig = px.histogram(rides_per_user, x='number of rides', nbins=5000, title='Distribution of Number of Rides per User')
fig.show()

In [ ]:
fig = px.box(rides_per_user, y="number of rides")
fig.show()

In [ ]:
# Distribution du nombre de mois actifs par conducteur

fig = px.histogram(months_per_user, x='number of months', nbins=30, title='Distribution of Number of Active Months per User')
fig.update_xaxes(tickmode='linear')
fig.show()

In [ ]:
fig = px.box(months_per_user, y="number of months")
fig.show()

In [ ]:
# Distribution du nombre de trajets mensuel moyen par conducteur

rides_per_month_per_user = pd.merge(rides_per_user, months_per_user, on='driver_uid')
rides_per_month_per_user['rides_per_user_per_month'] = rides_per_month_per_user['number of rides'] / rides_per_month_per_user['number of months']

fig = px.histogram(rides_per_month_per_user, x='rides_per_user_per_month', nbins=500, title='Distribution of Number of Average Monthly Rides per User')
fig.show()

In [ ]:
fig = px.box(rides_per_month_per_user, y="rides_per_user_per_month")
fig.show()

> 33% des conducteurs ont effectué un seul trajet, 50% ont effectué moins de 3 trajets, et 75% ont effectué moins de 7 trajets.

> A partir de ces statistiques, nous pouvons créer une métrique de performance par conducteur afin d'isoler les conducteurs fidèles ou ce qu'on peut appeler les top users. Ce type de users est important à détecter car un échange avec ces users peut nous donner des indications sur ce qui pourrait faire fidéliser le reste (c'est un segment déjà fidèle).

> Vu que 75% des conducteurs font moins de 7 trajets, sont actifs pendant uniquement 1 mois, et font moins de 6 trajets par mois en moyenne, nous pouvons considérer tout conducteurs qui se situe au dessus de ces valeurs et qui a été actif récemment un conducteur "fidèle" (nous pouvons mieux explorer la notion de rétention, i.e l'étalement dans le temps de l'activité par exemple, et créer ainsi une métrique mixte de "fidélisation" ou "non churn").

> On a également vu qu'il y a des users qui ont fait plus de 50 trajets par mois, cela vaudrait le coup de vérifier ce que ces conducteurs font et pourquoi (dans le chart ci-dessous, on voit que ces top users font aussi en majorité uniquement des trajets sans passagers, mais relativement à la distribution globale, ils font plus un mixe des deux que le reste des conducteurs).

In [ ]:
# Exemple de métrique de performance

driver_performance = routes.groupby('driver_uid').agg(
    total_trips = ('_id', 'nunique'),
    months = ('month', 'nunique'),
    avg_trip_length = ('length', 'mean'),
    last_trip = ('departure_day', 'max'),
)
driver_performance['completion_rate'] = routes[routes['status'] == 'DONE'].groupby('driver_uid')['driver_uid'].count() / driver_performance['total_trips']
driver_performance['avg_rides_per_month'] = round(driver_performance['total_trips'] / driver_performance['months'])

driver_performance[
    (driver_performance['total_trips'] > 7)
    & (driver_performance['last_trip'] > pd.Timestamp(pd.Timestamp('today') - relativedelta(months=3)).date())
    & (driver_performance['completion_rate'] > 0.9)
    & (driver_performance['months'] > 1)
    & (driver_performance['avg_rides_per_month'] > 6)
    ].sort_values(by=['completion_rate','total_trips'], ascending=False)

In [ ]:
# Visualisation du nomber quotidien de conducteurs par status du conducteur - Zoom sur top users

driver_performance['avg_rides_per_month'] = round(driver_performance['total_trips'] / driver_performance['months'])
top_users = pd.merge(driver_performance[driver_performance['avg_rides_per_month'] > 50], daily_activity, on='driver_uid', how='inner')

top_users = top_users.groupby(['departure_day','driver_status'])['driver_uid'].nunique().reset_index(name='number of users')
top_users.columns = ['departure day', 'driver status', 'number of users']

fig = px.bar(top_users, x='departure day', y='number of users', color='driver status', title='Number of Drivers by Day and Status', barmode='stack', category_orders={'driver status': ['Both','Rewarded','Non-Rewarded']})
fig.show()

> La part des conducteurs qui font au moins un trajet non récompensé est plus grande comparée à celle de la moyenne de tous les utilisateurs (le mauve + verts représentent une part relativement plus grande comparée à celle du chart fait sur tous les users).

### 3. Aquisition & rétention

In [ ]:
# Création du status du conducteur (new/retuning) et du nombre de jour après la première utilisation

routes_sorted = routes.sort_values(by=['driver_uid', 'departure_day'])
previous_values = routes_sorted[['driver_uid', 'departure_day']].drop_duplicates()
previous_values['previous_departure_day'] = previous_values.groupby('driver_uid')['departure_day'].shift(1)

# routes_sorted['previous_departure_day'] = routes_sorted.groupby('driver_uid')['departure_day'].shift(1)
routes_sorted = pd.merge(routes_sorted, previous_values, on=['driver_uid','departure_day'])

first_departure_day = routes_sorted.groupby('driver_uid')['departure_day'].min().reset_index(name='first_departure_day')

routes_sorted = pd.merge(routes_sorted, first_departure_day, on='driver_uid')
routes_sorted['days_post_start'] = pd.to_timedelta(routes_sorted['departure_day'] - routes_sorted['first_departure_day']).dt.days

routes_sorted['driver_status'] = 'New'
routes_sorted.loc[(routes_sorted['previous_departure_day'].notnull()) & (routes_sorted['previous_departure_day'] < routes_sorted['departure_day']), 'driver_status'] = 'Returning'
routes_sorted.drop(columns=['previous_departure_day'], inplace=True)

In [ ]:
# Visualisation du nombre quotidien de conducteurs par status du conducteur

users_per_status = routes_sorted.groupby(['departure_day','driver_status'])['driver_uid'].nunique().reset_index(name='number of users')
users_per_status.columns = ['departure day', 'driver status', 'number of users']

fig = px.bar(users_per_status, x='departure day', y='number of users', color='driver status', title='Number of Drivers by Day and Status', barmode='stack')
fig.show()

In [ ]:
# Visualisation du nombre quotidien de conducteurs par status du conducteur (100% stacked bars)

pivot_us = users_per_status.pivot(index='departure day', columns='driver status', values='number of users')
pivot_us = round(pivot_us.div(pivot_us.sum(axis=1), axis=0) * 100,2)
users_per_status_normalized = pivot_us.reset_index().melt(id_vars='departure day', var_name='driver status', value_name='percentage')

fig = px.bar(users_per_status_normalized, x='departure day', y='percentage', color='driver status', title='Number of Drivers by Day and Status (Stacked to 100%)', barmode='stack')
fig.show()

In [ ]:
# Visualisation du nomber quotidien de trajets par status du conducteur

rides_per_status = routes_sorted.groupby(['departure_day','driver_status'])['_id'].nunique().reset_index(name='number of rides')
rides_per_status.columns = ['departure day', 'driver status', 'number of rides']

fig = px.bar(rides_per_status, x='departure day', y='number of rides', color='driver status', title='Number of Rides by Day and Status', barmode='stack')
fig.show()

> 80% des conducteurs quotidiens sont des new, et le reste sont des returning. Ceci est plutôt positif et indique qu'il y a bien des utilisateurs "fidèles" (ce qui confirme ce qu'on avait conclu dans la dernière partie du paragraphe Daily usage - Users).

> En termes de trajets, il y a plus de trajets effectués par les returning (entre 4k et 6k trajets par jour) que par les new (entre 1.5k et 2.5k trajets par jour). Les returning font donc plus de trajets par conducteur que les new (les conducteurs font donc plus de trajets après le premier jour d'utilisation).

In [ ]:
# Distribution du nombre de trajets par jours post premier jour

users_per_days_post_start = routes_sorted.groupby(['days_post_start','driver_status'])['_id'].nunique().reset_index(name='number of rides')
users_per_days_post_start.columns = ['days post start day', 'driver status', 'number of rides']

fig = px.bar(users_per_days_post_start[users_per_days_post_start['driver status']=="Returning"], x='days post start day', y='number of rides', color='driver status', title='Number of Rides by Number of Days Post Start Day - Returning Users', barmode='stack')
fig.update_traces(marker_color='red')
fig.show()

In [ ]:
fig = px.histogram(routes_sorted[routes_sorted['driver_status']=="Returning"], x="days_post_start", histnorm='probability density', cumulative = True)
fig.update_traces(marker_color='red')
fig.show()

> ~23% des trajets des returning sont effectués dans les 5 jours post jour de début (le jour de début exclu), et 50% dont effectués dans les 10 jours post premier jour.

> On a une bonne contuinuité d'usage, donc un bon potentiel de rétention.

### 4. "Demand" & trip duration

In [ ]:
# Time-of-Day Analysis - test

# plt.figure(figsize=(12, 6))
# sns.histplot(routes['departure'].dt.hour, bins=24, color='blue', alpha=0.5, label='Departure')
# sns.histplot(routes['arrival'].dt.hour, bins=24, color='red', alpha=0.5, label='Arrival')
# plt.xlabel('Hour of Day')
# plt.ylabel('Frequency')
# plt.title('Distribution of Departure and Arrival Hours')
# plt.legend()
# plt.show()

In [ ]:
# Analyse des heures de pic

fig = go.Figure()
fig.add_trace(go.Histogram(x=routes['departure'].dt.hour, name='Departure', marker_color='blue', opacity=0.5))
fig.add_trace(go.Histogram(x=routes['arrival'].dt.hour, name='Arrival', marker_color='red', opacity=0.5))
fig.update_layout(barmode='overlay', xaxis_title='Hour of Day', yaxis_title='Frequency', title='Distribution of Departure and Arrival Hours')
fig.update_xaxes(tickmode='linear')
fig.show()

> Les arrivées et les départs semblent suivrent la même distribution par heure. Les trajets sont situés en majorité dans l'intervalle 6h-20h, avec un léger pic entre 15h et 17h. Il n'y a pas de pic particulier qui se distingue au global. Ce qui serait intéressant pour aller plus loin est de tracer la même distribution par pays et/ou par ville afin de voir s'il y a des patterns qui resortent.

In [ ]:
# Distribution des durées de trajet

fig = px.histogram(routes, x='trip_duration_min', nbins=5000, title='Distribution of Ride Durations in Minutes')
fig.show()

In [ ]:
fig = px.box(routes, y="trip_duration_min")
fig.show()

> Les durées des trajets sont dans leur majorité courts, avec 75% des trajets durant moins de 22 minutes, et la quasi majorité durant moins d'une heure. Ceci nous indique que les trajets sont -probablement- concentrés en Europe (naturellement au vu du marché de l'application, chose qu'on verra aussi dans les analyses géographiques du paragraphe V).

In [ ]:
# Analyse des durées de trajet par mois

fig = px.scatter(routes, x='departure_month', y='trip_duration_min', color='trip_duration_min', title='Trip Duration Distribution by Departue Month', labels={'departure_month': 'Departue Month', 'trip_duration_min': 'Trip Duration (min)'})
fig.update_layout(showlegend=False)
fig.update_xaxes(tickmode='linear')
fig.show()

In [ ]:
fig = px.box(routes, x="departure_month", y="trip_duration_min")
fig.update_xaxes(tickmode='linear')
fig.show()

> La distribution des durées des trajets varie selon le mois. Il y a plus de trajets longs en période d'hiver (décembre à février) et notamment en février, mais qui ne représentent pas la majorité. En effet, en termes de distributions, cette période constitue celle avec une majoirité de trajets les plus courts par rapport aux autres périodes. En parallèle, la période Mars-Juin connait une majorité de trajets plus longs.

>Voici un résumé de la distribution par mois:
> - Décembre - Janvier - Février  : 75% des trajets durent moins de `21 minutes`, avec une médiane autour de 11 minutes
> - Septembre - Octobre - Novembre: 75% des trajets durent moins de `24 minutes`, avec une médiane autour de 13 minutes
> - Juillet - Août                : 75% des trajets durent moins de `30 minutes`, avec une médiane autour de 17 minutes
> - Avril - Juin                  : 75% des trajets durent moins de `37 minutes`, avec une médiane autour de 25 minutes
> - Mars - Mai                    : 75% des trajets durent moins de `43 minutes`, avec une médiane autour de 26 minutes

In [ ]:
# Analyse des durées de trajet par heure

fig = px.scatter(routes, x=routes['departure_hour'], y='trip_duration_min', color='trip_duration_min', title='Trip Duration Distribution by Departure Hour', labels={'departure': 'Hour', 'trip_duration_min': 'Trip Duration (min)'})
fig.update_layout(showlegend=False)
# fig.update_xaxes(tickmode='linear')
fig.show()

In [ ]:
fig = px.box(routes, x="departure_hour", y="trip_duration_min")
fig.update_xaxes(tickmode='linear')
fig.show()

> La distribution des durées des trajets selon l'heure de départ est relativement plus uniforme. Il y a légèrement plus de trajets longs quand l'heure de départ est entre 2h et 5h, ce qui est logique (les longs trajets débuteront plus tôt le matin).

`❗il faut zoomer sur les graphes, sinon les boxplots sont écrasés par les valeurs extrêmes ❗`

# V. Geographical distribution

### 1. Geographical statistics

In [ ]:
# Analyse des trajets par ville de départ

# rides_per_d_city = origin_routes_geo.groupby(['departure_city'])['_id'].nunique().reset_index(name='number_of_rides')
rides_per_d_city = routes.groupby(['origin_city'])['_id'].nunique().reset_index(name='number_of_rides')
rides_per_d_city = rides_per_d_city.sort_values(by = ['number_of_rides'], ascending = False).head(20)
rides_per_d_city['percentage'] = rides_per_d_city['number_of_rides'] * 100/routes['_id'].nunique()

fig = px.bar(rides_per_d_city, x='number_of_rides', y='origin_city', color='origin_city', title='Number of Rides by Departure City', orientation = 'h')
for i, row in rides_per_d_city.iterrows():
    fig.add_annotation(
        x=row['number_of_rides'],  # x-coordinate position of the label
        y=row['origin_city'],  # y-coordinate position of the label
        text=f"{row['percentage']:.2f}%",  # text to display (percentage with two decimal places)
        showarrow=False,  # do not show arrow
        font=dict(color='black', size=10),  # font style
        xshift=5  # adjust the horizontal position of the label
    )
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

In [ ]:
# Analyse des trajets par ville d'arrivée

# rides_per_d_city = origin_routes_geo.groupby(['arrival_city'])['_id'].nunique().reset_index(name='number_of_rides')
rides_per_d_city = routes.groupby(['destination_city'])['_id'].nunique().reset_index(name='number_of_rides')
rides_per_d_city = rides_per_d_city.sort_values(by = ['number_of_rides'], ascending = False).head(20)
rides_per_d_city['percentage'] = rides_per_d_city['number_of_rides'] * 100/routes['_id'].nunique()

fig = px.bar(rides_per_d_city, x='number_of_rides', y='destination_city', color='destination_city', title='Number of Rides by Arrival City', orientation = 'h')
for i, row in rides_per_d_city.iterrows():
    fig.add_annotation(
        x=row['number_of_rides'],  # x-coordinate position of the label
        y=row['destination_city'],  # y-coordinate position of the label
        text=f"{row['percentage']:.2f}%",  # text to display (percentage with two decimal places)
        showarrow=False,  # do not show arrow
        font=dict(color='black', size=10),  # font style
        xshift=5  # adjust the horizontal position of the label
    )
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

In [ ]:
# Analyse des trajets par trajet effectué

routes['ride'] = routes['origin_city'] + "->" + routes['destination_city']

rides_per_d_city = routes.groupby(['ride'])['_id'].nunique().reset_index(name='number_of_rides')
rides_per_d_city = rides_per_d_city.sort_values(by = ['number_of_rides'], ascending = False).head(20)
rides_per_d_city['percentage'] = rides_per_d_city['number_of_rides'] * 100/routes['_id'].nunique()

fig = px.bar(rides_per_d_city, x='number_of_rides', y='ride', color='ride', title='Number of Rides by Route', orientation = 'h')
for i, row in rides_per_d_city.iterrows():
    fig.add_annotation(
        x=row['number_of_rides'],  # x-coordinate position of the label
        y=row['ride'],  # y-coordinate position of the label
        text=f"{row['percentage']:.2f}%",  # text to display (percentage with two decimal places)
        showarrow=False,  # do not show arrow
        font=dict(color='black', size=10),  # font style
        xshift=5  # adjust the horizontal position of the label
    )
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

In [ ]:
# Proposition d'ajout de données iso de ville, pays, et continent

# origin_routes_geo = routes[['origin_lat','origin_lng','_id']]
# destinsation_routes_geo = routes[['destination_lat','destination_lng','_id']]

In [ ]:
# origin_routes_geo['departure_city'] = origin_routes_geo.apply(lambda row: getcity(row['origin_lat'], row['origin_lng']), axis=1)
# origin_routes_geo['departure_country'] = origin_routes_geo.apply(lambda row: getcountry(row['origin_lat'], row['origin_lng']), axis=1)
# origin_routes_geo['departure_region'] = origin_routes_geo.apply(lambda row: getreg(row['origin_lat'], row['origin_lng']), axis=1)

In [ ]:
# destinsation_routes_geo['arrival_city'] = destinsation_routes_geo.apply(lambda row: getcity(row['destination_lat'], row['destination_lng']), axis=1)
# destinsation_routes_geo['arrival_country'] = destinsation_routes_geo.apply(lambda row: getcountry(row['destination_lat'], row['destination_lng']), axis=1)
# destinsation_routes_geo['arrival_region'] = destinsation_routes_geo.apply(lambda row: getreg(row['destination_lat'], row['destination_lng']), axis=1)

In [ ]:
# Analyse des trajets par pays de départ

# rides_per_d_city = origin_routes_geo.groupby(['departure_country'])['_id'].nunique().reset_index(name='number_of_rides')

# fig = px.bar(rides_per_d_city, x='number_of_rides', y='departure_country', color='departure_country', title='Number of Rides by Departure Country', orientation = 'h')
# fig.update_layout(yaxis={'categoryorder':'total ascending'})
# fig.show()

> Les top 20 villes de départ et d'arrivées sont toutes en France, avec Paris, Marseille, Lyon, et Amiens dans les top 4.

> Les trajets les plus fréquents sont intra-ville avec Paris-Paris, Marseille-Marseille dans le top 2.

### 2. Other geographical visualisations

In [ ]:
city_data = routes.groupby(['origin_city']).agg(
    trips = ('_id', 'nunique')
).reset_index()

city_data.columns = ['origin_city','trips']

fig = px.scatter_geo( city_data,
                      locations='origin_city',
                      locationmode='country names',
                      size='trips',
                      hover_name='origin_city',
                      projection='natural earth',
                      title='Intensity of Rides by City')
fig.update_layout(height=800, width=2000)
fig.show()

# fig = px.scatter_geo(city_data,
#                     lat = 'latitude',
#                     lon = 'longitude',
#                     size='trips',
#                     hover_name='origin_city',
#                     # geojson='geometry',
#                     title='Intensity of Departing Rides by City')
# fig.update_layout(height=800, width=2000)
# fig.show()

# VI. Correlation analysis

In [ ]:
CategoryGroupLists = routes.groupby('ride')['length'].apply(list)
AnovaResults = f_oneway(*CategoryGroupLists)
print('P-Value for Anova is: ', AnovaResults[1])

> La longueur du trajet dépend (évidemment) du trajet à effectuer (ville de départ et ville d'arrivée).

In [ ]:
CategoryGroupLists = routes.groupby('status')['reward_value'].apply(list)
AnovaResults = f_oneway(*CategoryGroupLists)
print('P-Value for Anova is: ', AnovaResults[1])

In [ ]:
fig = px.box(routes[routes["reward_currency"]=="EUR"], x="status", y="reward_value")
fig.show()

In [ ]:
fig = px.box(routes[routes["reward_currency"]=="EUR"], x="trip_duration_bin", y="reward_value")
fig.show()

> La valeur de la récompense est corrélée au status (il y a plus de récompense pour les trajets complétés qu'annulés, ce qui est rassurant, mais il faut investiguer les quelques outliers qui apparaissent. Pourquoi certains conducteur ont reçu une récompense de 50€ pour des trajets annulés?) et à la longueur du trajet. Les trajets très longs et très courts ont moins de récompenses.

# Take aways & next steps

---

#### **Take aways**

* `Data processing`
  * Cette partie est celle qui a pris le plus de temps. S'assurer que les données sont cohérentes et complètes est généralement la plus grande partie du travail

  * Nous avons constaté l'existance de valeurs (très) extêmes, notamment au niveau des longueurs (et donc durées) des trajets. Nous avons procédé à les garder vu que ces trajets sont théoriquement possibles. Cependant, pour des analyses plus ciblées, il peut être pertinent de les retirer (des points qui se décident généralement en coordination avec le business)
  
  * Voici quelques enrichissements possibles aux données existantes:
    * Création d'un ID de trajet consistant, dépendant de l'heure de départ et d'arrivée du trajet, l'ID du client, et les coorodnnées de départ et de destination. Ceci permettra par exepmple le problème des trajet qui ont le statut `CREATED` uniquement (la plupart étant des trajets qui ont été annulés mais dont l'ID a changé)
    * Nettoyage des données des trajets en faisant le lien entre les trajets sans "fin" et le reste
    * Normalisation des données de ville, région, pays, et continent, pour qu'ils soient sous un format iso et facile à visualiser


* `Usage`

  * Entre 4.5k et 6k trajets sont effectués chaque jour, et ce depuis février 2024. Le nombre de trajets quotidiens est en augmentation constante: On est passé de 5,710 trajets en moyenne par jour en janvier 2024, à 6,161 trajets en moyenne par jour en février 2024 (+8%), versus 3,640 en décembre 2023 (+69%)

  * Une grande partie des trajets quotidiens est finsalisée, notamment depuis Juillet 2023 (i.e le moment où le nombre de trajets a commencé à devenir significatif). Le nombre de trajets finalisés représente plus de 75% des trajets

  * Le nombre de trajets annulés a connu une légère augmentation depuis Novembre 2023, passant de ~20% à ~30% des trajets

  * Entre 80% et 92% des trajets (jusqu'à 7.5k trajets par jour) sont sans passager, ce qui est un grand axe de réflexion (est-ce un point saillant? si oui, comment encourager les utilisateurs à prendre des passagers?)

  * En termes de conducteurs, on a atteint ~3k conducteur par jour en février 2024. En zoomant sur le type de trajet : 85% à 90% des conducteurs quotidiens (2.5k à 3k conducteur par jour) n'effectuent que des trajets récompensés, i.e sans passager, 10% à 13% des conducteurs font les deux, et moins de 3% ne font que des trajets non récompensés.

  * 90% des trajets effectués sont courts (moins de 30 minutes). Voici quelques statistiques:

    * 40% à 50% des trajets (entre 3k et 3.7k trajets par jour) sont des trajets très courts (moins de 10 minutes)
    * ~ 40% des trajets (~ 3k trajets par jours) sont des trajets courts (entre 10 et 30 minutes)
    * 10% à 15% des trajets quotidiens (~ 700-800 trajets par jour) sont moyens (entre 30 minutes et 1 heure)
    * 2% à 3% des trajets (~150 trajets par jour) sont longs (entre 1 et 2 heures)
    * <= 3% des trajets (150 à 200 trajets par jour) sont très longs (plus de 2h, avec un maximum à 8,381 minutes = ~5 jours et 19 heures)

  * Vu que 75% des conducteurs font moins de 7 trajets, sont actifs pendant uniquement 1 mois, et font moins de 6 trajets par mois en moyenne, nous pouvons considérer tout conducteurs qui se situe au dessus de ces valeurs et qui a été actif récemment un conducteur "fidèle" (nous pouvons mieux explorer la notion de rétention, i.e l'étalement dans le temps de l'activité par exemple, et créer ainsi une métrique mixte de "fidélisation" ou "non churn")


* `Aquisition & rétention`

  * 80% des conducteurs quotidiens sont des new, et le reste sont des returning. Ceci est plutôt positif et indique qu'il y a bien des utilisateurs "fidèles" (ce qui confirme ce qu'on avait conclu dans la dernière partie du paragraphe Daily usage - Users)
  
  * En termes de trajets, il y a plus de trajets effectués par les returning (entre 4k et 6k trajets par jour) que par les new (entre 1.5k et 2.5k trajets par jour). Les returning font donc plus de trajets par conducteur que les new (en d'autres termes les conducteurs font plus de trajets après le premier jour d'utilisation que pendant le premier jour)

  * ~23% des trajets des returning sont effectués dans les 5 jours post jour de début (le jour de début exclu), et 50% dont effectués dans les 10 jours post premier jour. On a une bonne contuinuité d'usage, donc un bon potentiel de rétention sur les utilisateurs qui reviennent au moins une fois


* `Heures de pic`

  * Les arrivées et les départs semblent suivrent la même distribution par heure. Les trajets sont situés en majorité dans l'intervalle 6h-20h, avec un léger pic entre 15h et 17h. Il n'y a pas de pic particulier qui se distingue au global

  * La distribution des durées des trajets selon l'heure de départ est relativement plus uniforme. Il y a légèrement plus de trajets longs quand l'heure de départ est entre 2h et 5h, ce qui est logique (les longs trajets débuteront plus tôt le matin)

  * La distribution des durées des trajets varie selon le mois. Il y a plus de trajets longs en période d'hiver (décembre à février) et notamment en février, mais qui ne représentent pas la majorité (i.e il y a plus de valeurs extrêmes). En effet, en termes de distributions, cette période constitue celle avec une majoirité de trajets les plus courts par rapport aux autres périodes. En parallèle, la période Mars-Juin connait une majorité de trajets plus longs.

  * Voici un résumé de la distribution par mois:

    * Décembre - Janvier - Février : 75% des trajets durent moins de 21 minutes, avec une médiane autour de 11 minutes
    * Septembre - Octobre - Novembre: 75% des trajets durent moins de 24 minutes, avec une médiane autour de 13 minutes
    * Juillet - Août : 75% des trajets durent moins de 30 minutes, avec une médiane autour de 17 minutes
    * Avril - Juin : 75% des trajets durent moins de 37 minutes, avec une médiane autour de 25 minutes
    * Mars - Mai : 75% des trajets durent moins de 43 minutes, avec une médiane autour de 26 minutes


* `Distributions géographiques`

  * Les top 20 villes de départ et d'arrivées sont toutes en France, avec Paris, Marseille, Lyon, et Amiens dans les top 4

  * Les trajets les plus fréquents sont intra-ville avec Paris-Paris, Marseille-Marseille dans le top 2

#### **Pour aller plus loin**

* Ce serait intéressant de visualiser les trajets uniques (suite à la redéfinition d'un ID consistant de trajet unique) et voir quels trajets sont les plus effectués par mois par exemple (il se peut qu'il y ait des trajets plus fréquents pendant des périodes précises)

* Une visualisation géographique plus pratique peut être faite via Looker Studio ou Tableau. Les visualisations géographiques sur Google Colab ou Jupyter Notebook sont lourdes quand la taille des données dépasse un certain seuil (les outils de viz sont plus pratiques dans ces cas là)

* Des analyse prédictives peuvent être faites :

  * Prédiction de la longueur du trajet, de sa complétion (done ou canceled) selon les données liées au trajet et à l'historique du conducteur

  * Prédiction de la performance d'un conducteur (LTV, taux de complétion, rétention, etc) selon l'historique d'usage
  
  * etc

* Des analyses plus approfondies de corrélations peuvent être faites :

  * Corrélation entre la longueur d'un trajet et le croisement heure du jour x mois

  * Corrélation entre les départs / destinations et le status du trajet (récompensé ou pas)

  * Corrélation entre le status du trajet et le status du conducteur (new/returning)

  * Corrélation entre le nombre de trajets par conducteur et la ville de départ

  * Corrélation entre le nombre de trajets par conducteur et le mois

  * etc